In [74]:
import os
import nibabel as nib
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

slices_excluir = "/home/mariopasc/Python/Projects/BSC_final/epilepsy-displasia-focal-segmentation/text-info-files/excluir-slices.txt"

t2flair_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/T2flair-study"
t1w_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/T1w-study"
roi_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/roi"

t2flair_nii_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/T2flair-study-nii"
t1w_nii_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/T1w-study-nii"
roi_nii_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/roi-nii"

t2flair_im_train_path = "/home/mariopasc/Python/Datasets/t2flair-yolov8-ds/images/train"
t2flair_im_val_path = "/home/mariopasc/Python/Datasets/t2flair-yolov8-ds/images/val"
t2flair_label_train_path = "/home/mariopasc/Python/Datasets/t2flair-yolov8-ds/labels/train"
t2flair_label_train_val = "/home/mariopasc/Python/Datasets/t2flair-yolov8-ds/labels/val"

t1w_im_train_path = "/home/mariopasc/Python/Datasets/t1w-yolov8-ds/images/train"
t1w_im_val_path = "/home/mariopasc/Python/Datasets/t1w-yolov8-ds/images/val"
t1w_label_train_path = "/home/mariopasc/Python/Datasets/t1w-yolov8-ds/labels/train"
t1w_label_train_val = "/home/mariopasc/Python/Datasets/t1w-yolov8-ds/labels/val"

### Data Loader - Convertir .nii.gz a imágenes PNG o JPEG2000

In [79]:
# Esta función convierte un estudio nii.gz a diversas imágenes .nii, asociando como nombre de las imágenes idpaciente-slice.nii.
# VER2: Se añade también un parámetro de entrada que es una lista de rodajas que no se deben incluir
def convert_gz_nii(niigz_file, save_path, path_excluir, patient_id):
    # Verificar si el archivo .nii.gz existe
    if not os.path.isfile(niigz_file):
        print("Error: Specified .nii.gz file doesn't exist.")
        return
    # Creamos el vector con las rodajas a exlcuir
    exclude_slices = []
    # Si existe el archivo .txt, rellenamos el vector con las rodajas que hay que excluir
    # Si no existe, se asume que se incluyen todas. 
    if os.path.isfile(path_excluir):
        with open(path_excluir, 'r') as archivo:
            for linea in archivo:
                exclude_slices.append(int(linea.strip()))
    
    # Cargar el archivo .nii.gz
    img = nib.load(niigz_file)
    data = img.get_fdata()
    # Obtener el número de rodajas
    num_slices = data.shape[2]

    # Crear el directorio de destino si no existe
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Iterar sobre todas las rodajas y guardarlas como imágenes .nii
    for i in range(num_slices):
        # Verificar si la rodaja está en la lista de exclusión
        if i in exclude_slices:
            continue  # Saltar esta rodaja

        # Obtener una sola rodaja
        slice_data = data[:, :, i]
        # Crear el nombre de la imagen
        image_name = f"{patient_id}-{i}.nii"
        # Guardar la imagen .nii
        nib.save(nib.Nifti1Image(slice_data, img.affine), os.path.join(save_path, image_name))

    print("Process finished with exit code 0")


# Esta función hace uso de la anterior para convertir un estudio completo a formato .nii
def study_to_nii(study_path, save_path, path_excluir):
    if not os.path.exists(study_path):
        print("Input file does not exist")
        return
    
    # Por cada paciente dentro del estudio
    for patient_id in os.listdir(study_path):
        # Comprobar si el elemento es una carpeta
        patient_folder = os.path.join(study_path, patient_id)
        if not os.path.isdir(patient_folder):
            print(f"Patient {patient_folder} not found")
            continue

        # Extraemos el nombre de su archivo nii.gz
        niigz_files = [path for path in os.listdir(patient_folder) if path.endswith(".nii.gz")]
        if not niigz_files:
            print(f"No .nii.gz file found for patient {patient_id}")
            continue

        # Asumimos que solo hay un archivo .nii.gz por paciente
        niigz_file = os.path.join(patient_folder, niigz_files[0])
        convert_gz_nii(niigz_file=niigz_file, save_path=save_path, patient_id=patient_id, path_excluir=path_excluir)

# Esta función toma como entrada una carpeta con imágenes .nii y las guarda en la carpeta de destino en el formato especificado. 
def convert_nii_image_holdout(input_txt):
    # Verificar si el archivo .txt existe
    if not os.path.isfile(input_txt):
        print("Input txt file does not exist.")
        return
    
    # Leer el archivo .txt
    with open(input_txt, 'r') as file:
        lines = file.readlines()
    
    # Obtener las rutas de los folders y archivos
    nii_folder = lines[0].split(':')[1].strip()
    train_folder = lines[1].split(':')[1].strip()
    train_nii_files = lines[2].split(':')[1].strip()
    val_folder = lines[3].split(':')[1].strip()
    val_nii_files = lines[4].split(':')[1].strip()
    test_folder = lines[5].split(':')[1].strip()
    test_nii_files = lines[6].split(':')[1].strip()
    imformat = lines[7].split(':')[1].strip().lower()
    
    # Función para convertir .nii a imagen
    def convert_to_image(nii_files, folder):
        for nii_file in nii_files:
            # Cargar el archivo .nii
            img = nib.load(os.path.join(nii_folder, nii_file))
            data = img.get_fdata()

            # Verificar si hay valores no válidos en los datos
            if np.any(np.isnan(data)) or np.any(np.isinf(data)):
                print(f"Skipping {nii_file}: Invalid values encountered.")
                continue
            
            # ========== NORMALIZACIÓN PROVISIONAL ==========
            # Normalizar los valores de píxel para que estén entre 0 y 255
            data_normalized = ((data - np.min(data)) / (np.max(data) - np.min(data)) * 255).astype(np.uint8)
            # ========== FIN NORMALIZACIÓN PROVISIONAL ======
            
            # Obtener el nombre del archivo sin la extensión .nii
            file_name, _ = os.path.splitext(nii_file)
            # Crear el nombre de la imagen con el formato especificado
            image_name = f"{file_name}.{imformat}"
            # Guardar la imagen en el formato especificado
            cv.imwrite(os.path.join(folder, image_name), data_normalized)

    # Obtener los nombres de archivos .nii para train, val y test
    train_nii_files = [file.strip() for file in open(train_nii_files, 'r').readlines()]
    val_nii_files = [file.strip() for file in open(val_nii_files, 'r').readlines()]
    test_nii_files = [file.strip() for file in open(test_nii_files, 'r').readlines()]
    
    # Convertir .nii a imágenes para train, val y test
    convert_to_image(train_nii_files, train_folder)
    convert_to_image(val_nii_files, val_folder)
    convert_to_image(test_nii_files, test_folder)

    print("Process finished with exit code 0.")

# Esta función se encarga del hold out (gracias a dios que existe scikit-learn)
def holdout_nii_images(folder_path, val_percent, test_percent, output_path):
    # Obtener la lista de archivos .nii en la carpeta
    nii_files = [file for file in os.listdir(folder_path) if file.endswith('.nii')]

    # Dividir los nombres de los archivos en train, val y test
    train_files, val_test_files = train_test_split(nii_files, test_size=(val_percent + test_percent), random_state=42)
    val_files, test_files = train_test_split(val_test_files, test_size=test_percent/(val_percent + test_percent), random_state=42)

    # Escribir los nombres de los archivos en archivos de texto
    def write_to_txt(file_list, txt_path):
        with open(txt_path, 'w') as file:
            for file_name in file_list:
                file.write(file_name + '\n')

    write_to_txt(train_files, os.path.join(output_path, 'train_files.txt'))
    write_to_txt(val_files, os.path.join(output_path, 'val_files.txt'))
    write_to_txt(test_files, os.path.join(output_path, 'test_files.txt'))

    print("Hold-out completed successfully.")



Hold-out completed successfully.


Workflow actual: 

1. Convertir el estudio dado a .nii. La carpeta del estudio dado debe tener en su interior carpetas con el nombre del paciente "sub-00XXXX" y, dentro de esas carpetas, debe estar el archivo .nii.gz. Se puede dar como argumento un .txt con rodajas que se excluirán de convertir a .nii. 
2. Realizar el hold-out de los datos. Los archivos .nii dedicados a train, val y test serán guardados en 3 archivos .txt. Estos archivos serán recibidos como entrada en la función que convierte los archivos .nii a PNG (paso 3) y las redirigirá a sus respectivas carpetas. 
3. Convertir el estudio en .nii a un formato sin pérdidas. Se ha implementado una normalización provisional dentro de este método. En un futuro, cuando se aplique un preprocesamiento a las rodajas del archivo .nii.gz, esta normalización ya vendrá dada. 

In [83]:
holdout_nii_images(folder_path=t2flair_nii_path,
                   val_percent=0.3,
                   test_percent=0.1,
                   output_path="/home/mariopasc/Python/Projects/BSC_final/epilepsy-displasia-focal-segmentation/text-info-files/t2flair-holdout")
    
convert_nii_image_holdout(input_txt="/home/mariopasc/Python/Projects/BSC_final/epilepsy-displasia-focal-segmentation/text-info-files/t2study-specifications.txt")

Process finished with exit code 0.


ERROR (1): Parece que ha habido un problema con el nombre, hay pacientes que sí tienen roi pero que el nombre de su fichero T2 contiene solo la palabra FLAIR
SOLUCIÓN (1): Se ha solucionado al cambiar una línea en el ejecutable organizar_estudios.sh: 
```find "$folder/anat" -type f -name "*FLAIR*" -exec mv -t "$destination_t2flair_dir/$folder_name" {} +``` 


In [77]:
# Vamos a comprobar que funciona...
def count_files_in_txt(txt_file):
    try:
        with open(txt_file, 'r') as file:
            file_count = sum(1 for line in file)
        return file_count
    except FileNotFoundError:
        print(f"El archivo {txt_file} no existe.")
        return None

# Ejemplo de uso:
txt_file_path_1 = "/home/mariopasc/Python/Projects/BSC_final/epilepsy-displasia-focal-segmentation/text-info-files/train_files.txt"
file_count1 = count_files_in_txt(txt_file_path_1)
txt_file_path_2 = "/home/mariopasc/Python/Projects/BSC_final/epilepsy-displasia-focal-segmentation/text-info-files/val_files.txt"
file_count2 = count_files_in_txt(txt_file_path_2)
txt_file_path_3 = "/home/mariopasc/Python/Projects/BSC_final/epilepsy-displasia-focal-segmentation/text-info-files/test_files.txt"
file_count3 = count_files_in_txt(txt_file_path_3)

print(f"La cantidad de archivos TRAIN es: {file_count1}")
print(f"La cantidad de archivos VAL es: {file_count2}")
print(f"La cantidad de archivos TEST es: {file_count3}")

total = file_count1 + file_count2 + file_count3
print(f"Porcentajes: \n train: {file_count1/total * 100}% \n val: {file_count2/total * 100}% \n test: {file_count3/total * 100}%")

La cantidad de archivos TRAIN es: 4773
La cantidad de archivos VAL es: 2387
La cantidad de archivos TEST es: 796
Porcentajes: 
 train: 59.99245852187028% 
 val: 30.002513826043238% 
 test: 10.005027652086476%


Como queremos exlcuir algunas rodajas se ha creado esta función para guardar las rodajas a excluir

In [ ]:
def guardar_vector_en_txt(path_salida, vector):
    with open(path_salida, 'w') as archivo:
        for valor in vector:
            archivo.write(str(valor) + '\n')

vector = list(range(1, 121)) + list(range(223, 257))
guardar_vector_en_txt(os.path.join("/home/mariopasc/Python/Projects/BSC_final/epilepsy-displasia-focal-segmentation/text-info-files", "excluir-slices.txt"),
                      vector)

Se preguntó por el procedimiento que se debe tener a la hora de tratar con cortes sin información. Por ahora se va a implementar una función que devuelva en un fichero .txt todos los cortes del paciente sub-00XXX que no contengan un ROI. 

In [19]:
# Esta función guarda en un .txt las rodajas de un estudio .nii.gz que no tienen un ROI asociado. 
def find_empty_slices(input_path, output_path):
    # Verificar si el directorio de entrada existe
    if not os.path.isdir(input_path):
        print("Input directory does not exist.")
        return
    if os.path.isdir(output_path):
        # Si no existe el .txt lo creamos con el nombre idpaciente-noROI.txt
        output_path = os.path.join(output_path, (os.path.split(input_path)[-1] + "-noROI.txt"))

    # Obtener la lista de archivos .nii.gz en el directorio de entrada
    nii_files = [file for file in os.listdir(input_path) if file.endswith('.nii.gz')]

    empty_slices = []

    for nii_file in nii_files:
        # Cargar el archivo .nii.gz
        img = nib.load(os.path.join(input_path, nii_file))
        data = np.uint8(img.get_fdata())
        num_slices = data.shape[2]

        for i in range(num_slices):
            # Obtener una sola rodaja
            slice_data = data[:, :, i]

            # Encontrar contornos en la imagen
            contours, _ = cv.findContours(slice_data, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

            # Si no hay contornos en la rodaja, agregar el número de corte a la lista
            if len(contours)==0:
                empty_slices.append(i)

    # Escribir la lista de cortes vacíos en el archivo de salida
    with open(output_path, 'w') as file:
        for slice_num in empty_slices:
            file.write(f"{slice_num}\n")

    print("Process finished with exit code 0")


Process finished with exit code 0


No es el mejor acercamiento, hay muy pocas rodajas con ROI asociado. Se va a optar por un enfoque semi-automático, en el que se quitarán las slices pasadas como parámetro. 